In [ ]:
%load_ext jupyternotify
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
from dedalus.extras import flow_tools
import time
import requests
from scipy.special import erf
from dedalus.core.operators import GeneralFunction

plt.rcParams['text.usetex'] = True


### Domain 

Lx = 1.0e7 
Ly = 1.0e7 
xmin, xmax = -Lx/2, Lx/2
ymin, ymax = -Ly/2, Ly/2
Nx, Ny = 128, 128 # Resolution

f0 = 0
f_ = 4.0e-4 # For f-plane 
beta = 2.0e-11 # For beta plane 

### Simulation Parameters 
g = 10.0 #g
H = 30.0 #Mean height
tau = 900.0 #Condensation timescale
tau_r = 2*(60*60*24) # radiative relaxation timescale. "is of the order of a few days" VP2020 
# lambda_r = 1.1e-5 #lambda_r as written in VP reply
tau_e = 1e6 #evaporative timescale, analagous to lambda_, from VP reply
lambda_ =0.08 #11*(60*60*24) #evaporation coefficient, "with no velocity dependence the system becomes saturated within a few days."
h_0 = 0 #VP2020

q_0 = 1.0 # "Taken to be unity"
q_g = 1.0 # 

L = 2.4*10**6 # Latent heat of condensation
Q_a = 0.035 # Tropical surfaace saturated specific humidy
c_p = 1004 # c_p 
T_0 = 300 
Rv = 462 # Water vapor gas constant
 
gamma = 8.5 #  (L*H*Q_a)/ (q_0*c_p*T_0) # ~8.5 in VP2020, 5 in reply
alpha = 60# L / (Rv*T_0) # ~20 in VP2020, 2 in reply

nu_u =1.0e4 # Viscosities, VP 2020 
nu_h =1.0e4 
nu_q = 2.0e4

/Users/luitbald/miniforge3/envs/dedalus3/lib/python3.12/site-packages/jupyternotify/jupyternotify.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


<IPython.core.display.Javascript object>

In [ ]:
# Dedalus setup

coords = d3.CartesianCoordinates('x', 'y')
dist = d3.Distributor(coords, dtype = np.float64)
x_basis = d3.RealFourier(coords['x'] , size = Nx, bounds = (xmin, xmax), dealias=3/2)
y_basis = d3.Chebyshev(coords['y'] , size = Ny, bounds = (ymin, ymax), dealias=3/2)
tau_basis = y_basis.derivative_basis(1)
x = dist.local_grid(x_basis)
y = dist.local_grid(y_basis)

U = dist.VectorField(coords, name='U', bases=(x_basis,y_basis))
h = dist.Field(name = 'h', bases = (x_basis, y_basis))
q = dist.Field(name = 'q', bases = (x_basis, y_basis))
f = dist.Field(name='f', bases=(x_basis,y_basis))

# f['g'] = f_ # f-plane
f['g'] = f0 + beta*y  #beta plane

grad = lambda A: d3.Gradient(A)
lap = lambda A: d3.Laplacian(A)
div = lambda A: d3.Divergence(A)
dx = lambda A: d3.Differentiate(A, coords['x'])
dy = lambda A: d3.Differentiate(A, coords['y'])
zcross = lambda A: d3.Skew(A)
trace = lambda A: d3.Trace(A)
dot = lambda A,B: d3.DotProduct(A,B)

k_heaviside = 10e5

HeavisideTheta = lambda x: 0.5 + 0.5* erf(k_heaviside*x)
lift = lambda A: d3.Lift(A, tau_basis, -1)
norm = lambda A: np.sqrt(A@A)

In [8]:
#tau terms for rigid BCs

tau_Uy_1 = dist.VectorField(coords, name='tau_Uy_1', bases=(x_basis))
tau_Uy_2 = dist.VectorField(coords,name='tau_Uy_2', bases=(x_basis))

# tau_Ux_1 = dist.VectorField(coords,name='tau_Ux_1', bases=(x_basis,))
# tau_Ux_2 = dist.VectorField(coords,name='tau_Ux_2', bases=(x_basis,))

tau_h_1 = dist.Field(name='tau_h_1', bases=(x_basis, ))
tau_h_2 = dist.Field(name='tau_h_2', bases=(x_basis,))

tau_q_1 = dist.Field(name='tau_q_1', bases=(x_basis,))
tau_q_2 = dist.Field(name='tau_q_2', bases=(x_basis,))



In [ ]:

E = lambda A:  HeavisideTheta(q_g - A) * (q_g - A) / tau_e # No velocity-dependence
q_sat = lambda h_: q_0 * d3.exp(-1*alpha * h_ / H) # Saturation specific humidity
C = lambda h_,q_: HeavisideTheta(q_-q_sat(h_))*(q_-q_sat(h_))/tau # condensation

problem = d3.IVP([U, h, q, tau_Uy_1, tau_Uy_2, tau_h_1, tau_h_2, tau_q_1, tau_q_2], namespace=locals()) 

ex, ey = coords.unit_vector_fields(dist) 
U_x = U @ ex # x velocity component
U_y = U @ ey # y velocity component 
grad_u =d3.grad(U) + ey*lift(tau_Uy_1) # first-order reduction, U
grad_h = d3.grad(h) + ey*lift(tau_h_1) # first-order reduction, h
grad_q = d3.grad(q) + ey*lift(tau_q_1) # first-order reduction, q
u_dot_grad_q= d3.DotProduct(U, grad_q)  # not used
dy_Ux = d3.Differentiate(U_x, coords['y'])  # shear

# No advective terms
problem.add_equation("dt(U) + f*zcross(U) + g*grad_h + lift(tau_Uy_2)-nu_u*div(grad_u)= 0")
problem.add_equation("dt(h) + H*trace(grad_u) -(h_0-h)/tau_r + lift(tau_h_2) - nu_h*div(grad_h) = -1*gamma*C(h,q)")
problem.add_equation("dt(q) + lift(tau_q_2) -nu_q*div(grad_q)  = E(q)-C(h,q) - q*div(U) - dot(U, grad(q))")

### Free-sliip/rigid boundaries
problem.add_equation("(U_y)(y = 'left')= 0")
problem.add_equation("(U_y)(y = 'right')= 0")
problem.add_equation("dy_Ux(y = 'left') = 0")
problem.add_equation("dy_Ux(y = 'right')= 0")
problem.add_equation("dy(h)(y='right')=0")
problem.add_equation("dy(h)(y='left')=0")
problem.add_equation("dy(q)(y='right')=0")
problem.add_equation("dy(q)(y='left')=0")

solver = problem.build_solver('RK222') 

2025-11-17 15:42:38,937 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 2.4e+00/s


In [ ]:
# Initial Conditions

x = dist.local_grid(x_basis, scale = 1)
y = dist.local_grid(y_basis, scale= 1)

gaussian = lambda x,y,sigma: d3.exp(-0.5*((x-1000)**2 + y**2) / sigma**2)
h['g'] = 0.2*gaussian(x,y,0.05*np.sqrt(Lx*Ly))# Gaussian bump

U['g'][0] = 0.0
U['g'][1] = 0.0
q['g'] =0.9*gaussian(x,y,0.05*np.sqrt(Lx*Ly)) # Gaussian bump  
# q['g'] = 0.0     

### Random Noise 
# rng = np.random.default_rng(42)                
# h_noise = rng.normal(loc=0.0, scale=1.0, size=h['g'].shape).astype(h['g'].dtype)
# h['g'] += 0.0005 * h_noise



In [6]:

u_list = []
h_list = []
t_list = []
C_list = []
E_list = []
q_list = []
init_timestep = 0.1

In [ ]:
%%notify -m "SIMULATION COMPLETE -- making animation... "

import logging
logger = logging.getLogger(__name__)

solver.stop_sim_time = 24*60*60*30

CFL = flow_tools.CFL(solver, initial_dt=init_timestep, cadence=10, safety=0.3, max_change=1.5)

CFL.add_velocity(U) 

logger.info('Starting loop')
start_time = time.time()
while solver.proceed:
    timestep = np.minimum(CFL.compute_timestep(), tau*1.2) #minimum of CFL condition, factor of condensation timescale 
    solver.step(timestep)
    
    if solver.iteration % 10 == 0:
        #Save data every 10th iteration
        t_list.append(solver.sim_time)
        # u_list.append(np.copy(U['g'][0]))
        h_list.append(np.copy(h['g']))
        # q_list.append(np.copy(q['g']))
        # q_sat_list.append(np.copy(q_sat(h)['g']))
        # E_list.append(E(q)['g'])
        C_list.append(C(h,q)['g'])

        print('Completed iteration {}, time {} days, dt {}'.format(solver.iteration, t_list[-1]/60/60/24, timestep))

end_time = time.time()
logger.info('Run time: %f' %(end_time-start_time))
logger.info('Iterations: %i' %solver.iteration)
requests.post("https://ntfy.sh/LEOPIDOLITE_SIMS", data = "SIMULATION COMPLETE -- making animation".encode(encoding = 'utf-8'))

2025-11-17 15:35:06,252 __main__ 0/1 INFO :: Starting loop
Completed iteration 10, time 1.1574074074074073e-05 days, dt 0.1
Completed iteration 20, time 2.8356481481481473e-05 days, dt 0.15000000000000002
Completed iteration 30, time 5.353009259259259e-05 days, dt 0.22500000000000003
Completed iteration 40, time 9.129050925925929e-05 days, dt 0.3375
Completed iteration 50, time 0.00014793113425925925 days, dt 0.5062500000000001
Completed iteration 60, time 0.00023289207175925916 days, dt 0.7593750000000001
Completed iteration 70, time 0.0003603334780092593 days, dt 1.1390625
Completed iteration 80, time 0.0005514955873842591 days, dt 1.7085937500000001
Completed iteration 90, time 0.0008382387514467589 days, dt 2.562890625
Completed iteration 100, time 0.0012683534975405093 days, dt 3.8443359375000004
Completed iteration 110, time 0.001913525616681134 days, dt 5.7665039062500005
Completed iteration 120, time 0.0028812837953920718 days, dt 8.649755859375
Completed iteration 130, time 0.

/Users/luitbald/miniforge3/envs/dedalus3/lib/python3.12/site-packages/dedalus/core/operators.py:613: RuntimeWarning: overflow encountered in exp
  self.func(arg0.data, out=out.data)


Completed iteration 250, time 0.4972636912967318 days, dt 1080.0
Completed iteration 260, time 0.6222636912967318 days, dt 1080.0
Completed iteration 270, time 0.7472636912967318 days, dt 1080.0
Completed iteration 280, time 0.8722636912967318 days, dt 1080.0
Completed iteration 290, time 0.9972636912967318 days, dt 1080.0
Completed iteration 300, time 1.1222636912967319 days, dt 1080.0
Completed iteration 310, time 1.2472636912967319 days, dt 1080.0
Completed iteration 320, time 1.3722636912967319 days, dt 1080.0
Completed iteration 330, time 1.4972636912967319 days, dt 1080.0
Completed iteration 340, time 1.6222636912967319 days, dt 1080.0
Completed iteration 350, time 1.7472636912967319 days, dt 1080.0
Completed iteration 360, time 1.8722636912967319 days, dt 1080.0
Completed iteration 370, time 1.9972636912967319 days, dt 1080.0
Completed iteration 380, time 2.1222636912967316 days, dt 1080.0


KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [ ]:
%%notify -m "Animation Complete -- MJO_VALLIS"
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import numpy as np
h_data = np.array(h_list)
t_data = np.array(t_list)
C_data = np.array(C_list)

frame_skip = 3 # every 30 frames plotted
h_anim = h_data[::frame_skip]
C_anim = C_data[::frame_skip]
t_anim = t_data[::frame_skip]
n_frames = len(t_anim)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

h0 = h_anim[0].T
C0 = C_anim[0].T
h_vmax0 = np.max(np.abs(h0))
im1 = ax1.imshow(h0,extent=[xmin, xmax, ymin, ymax],origin='lower', map='RdBu_r',  vmin=-h_vmax0, vmax=h_vmax0)
cbar1 = fig.colorbar(im1, ax=ax1, label='Height (h)')

ax1.set_xlabel('x')
ax1.set_ylabel('y')

C_vmin0, C_vmax0 = np.min(C0), np.max(C0)
im2 = ax2.imshow(C0, extent=[xmin, xmax, ymin, ymax],  origin='lower',cmap='Blues', vmin=C_vmin0, vmax=C_vmax0)
cbar2 = fig.colorbar(im2, ax=ax2, label='Rainfall (C)')
ax2.set_xlabel('x')
title = fig.suptitle(f'Time = {t_anim[0]/(24*60*60):.2f} Days')

plt.tight_layout()

def animate(i):
    h_frame = h_anim[i].T
    C_frame = C_anim[i].T
    im1.set_data(h_frame)
    im2.set_data(C_frame)

    h_vmax = np.max(np.abs(h_frame))
    if h_vmax == 0: h_vmax = 1e-12
    im1.set_clim(-h_vmax, h_vmax)
    cbar1.update_normal(im1)  

    C_vmin, C_vmax = np.min(C_frame), np.max(C_frame)
    if C_vmin == C_vmax:
        C_vmin -= 1e-12
        C_vmax += 1e-12
    im2.set_clim(C_vmin, C_vmax)
    cbar2.update_normal(im2)

    title.set_text(f'Time = {t_anim[i]/(24*60*60):.2f} Days')

    return im1, im2, title

ani = FuncAnimation(fig, animate, frames=n_frames, blit=False)
plt.close(fig)

#save animation
requests.post("https://ntfy.sh/LEOPIDOLITE_SIMS", data="ANIMATION COMPLETE".encode('utf-8'))
outname = f'/Users/luitbald/CODE/movies/MJO/MJO_VALLIS_TESTING_{datetime.now().strftime("%y-%m-%d %H:%M")}.mp4'
ani.save(outname, writer='ffmpeg', fps=30, dpi=150)


2025-11-17 14:58:28,340 matplotlib.animation 0/1 INFO :: Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2025-11-17 14:58:28,341 matplotlib.animation 0/1 INFO :: MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 2400x900 -pix_fmt rgba -framerate 30 -i pipe: -vcodec h264 -pix_fmt yuv420p -y '/Users/luitbald/CODE/movies/MJO/MJO_VALLIS_TESTING_25-11-17 14:58.mp4'


<IPython.core.display.Javascript object>